In [ ]:
import json

import boto3
import geopandas as gpd
import holoviews as hv
import hvplot.xarray  # noqa: F401
from dea_tools.spatial import points_on_line
from holoviews import opts
from odc.stac import load
from pystac import Item
from shapely.geometry import LineString

In [ ]:
# This is south west Tasmania
tiles = [
    # ("x038","y008"),
    # ("x038","y007"),
    # ("x038","y006"),  # Doesn't exist
    # ("x039","y008"),
    # ("x039","y007"),
    # ("x039","y006"),
    # ("x040","y008"),
    ("x040","y007"),
    # ("x040","y006"),
]

items = []
s3 = boto3.client('s3')

for tile in tiles:
    s3_url = (
        "s3://easihub-csiro-dc-data-projects/sar-capability/"
        f"csiro_s1_mosaic_annual_20m/0.3.0/{tile[0]}/{tile[1]}/2023--P1Y/"
        f"csiro_s1_mosaic_annual_20m_{tile[0]}{tile[1]}_2023--P1Y.stac-item.json"
    )
    # Open the S3 item as a dict
    s3_object = s3.get_object(Bucket=s3_url.split('/')[2], Key='/'.join(s3_url.split('/')[3:]))
    item = json.loads(s3_object['Body'].read().decode('utf-8'))
    items.append(Item.from_dict(item))

In [ ]:
loaded = load(items, chunks={})
loaded

In [ ]:
VARIABLE = "vv_gm"

# RGB image/map
map = loaded[VARIABLE].hvplot.image("x", "y", cmap="viridis", width=800, height=600, clim=(0, 0.2))

# Set up a holoviews points array to enable plotting of the clicked points
xthird = loaded.x.values[int(len(loaded.x) / 3)]
xsecondthird = loaded.x.values[int(len(loaded.x) / 3 * 2)]
ymid = loaded.y.values[int(len(loaded.y) / 2)]

first_points = (
    [xthird, xsecondthird], [ymid, ymid], [0, 1]
)
points = hv.Points(first_points, vdims='id')

points_stream = hv.streams.PointDraw(
    data=points.columns(),
    source=points,
    drag=True,
    num_objects=2,
    styles={'fill_color': "orange", 'line_color': 'gray'}
)

# clickxy = hv.streams.Tap(source=map, x=xthird, y=ymid)

def interpolate_points(data) -> gpd.GeoDataFrame:
    line_string = LineString(zip(data["x"], data["y"]))
    line_gdf = gpd.GeoDataFrame(geometry=[line_string,])
    
    # Interpolate more points
    return points_on_line(line_gdf, 0, 20)


# Function to build spectral plot of clicked location to show on hover stream plot
def click_profile(data):
    interp_points = interpolate_points(data)
    points_xarray = interp_points.assign(x=interp_points.geometry.x, y=interp_points.geometry.y).to_xarray()

    values = loaded.sel(points_xarray[['x', 'y']], method="nearest").squeeze()
    line_plot = values[VARIABLE].hvplot.line(color="orange")

    return hv.Overlay([line_plot])


def update_line(data):
    line_points = hv.Points(data, vdims='id')
    dynamic_line = hv.Curve(line_points, vdims="id").opts(color='orange')

    return hv.Overlay([dynamic_line])

# Define the Dynamic Maps
plot_profile_dmap = hv.DynamicMap(click_profile, streams=[points_stream])
map_profile_dmap = hv.DynamicMap(update_line, streams=[points_stream])

# Plot the Map and Dynamic Map side by side
hv.Layout(map * points * map_profile_dmap + plot_profile_dmap).cols(1).opts(
    opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
    opts.Labels(xoffset=0.01, yoffset=0.01, bgcolor='gray', text_color='black'),
    opts.Overlay(show_legend=False, show_title=False, frame_width=600)
)